<a href="https://colab.research.google.com/github/mamigal/dw_matrix_road_sign/blob/master/day5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install hyperopt

In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from skimage import color, exposure
import os
import datetime

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D, Dense, Flatten, Dropout
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import accuracy_score

%reload_ext tensorboard

from hyperopt import hp, STATUS_OK, tpe, Trials, fmin

In [3]:
cd '/content/drive/My Drive/Colab Notebooks/matrix/matrix_three/dw_matrix_road_sign'

/content/drive/My Drive/Colab Notebooks/matrix/matrix_three/dw_matrix_road_sign


In [0]:
train = pd.read_pickle('data/train.p')
test = pd.read_pickle('data/test.p')

X_train, y_train = train['features'], train['labels']
X_test, y_test = test['features'], test['labels']

df = pd.read_csv('data/signnames.csv')
labels_dict = df.to_dict()['b']

In [0]:
if y_train.ndim == 1: y_train = to_categorical(y_train)
if y_test.ndim == 1: y_test = to_categorical(y_test)

In [0]:
input_shape = X_train.shape[1:]
num_classes = y_train.shape[1]

In [0]:
def train_model(model, X_train, y_train, params_fit={}):
  model.compile(loss= 'categorical_crossentropy', optimizer='Adam', metrics='accuracy')

  logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
  tensorboard_callback = tf.keras.callbacks.TensorBoard( logdir, histogram_freq=1)
  
  model.fit(
      X_train, 
      y_train,
      batch_size=params_fit.get('batch size', 128),
      epochs=params_fit.get('epochs', 5),
      verbose=params_fit.get('verbose', 1),
      validation_data=params_fit.get('validation data', (X_train, y_train)),
      callbacks=[tensorboard_callback]
  )
  return model

def predict(model_trained, X_test, y_test, scoring=accuracy_score):
  y_test_norm = np.argmax(y_test, axis=1)

  y_pred_prob = model_trained.predict(X_test)
  y_pred = np.argmax(y_pred_prob, axis=1)

  return scoring(y_test_norm, y_pred)

In [18]:
  model.compile(loss= 'categorical_crossentropy', optimizer='Adam', metrics='accuracy')

  logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
  tensorboard_callback = tf.keras.callbacks.TensorBoard( logdir, histogram_freq=1)
  
  model.fit(
      X_train, 
      y_train,
      batch_size=params_fit.get('batch size', 128),
      epochs=params_fit.get('epochs', 5),
      verbose=params_fit.get('verbose', 1),
      validation_data=params_fit.get('validation data', (X_train, y_train)),
      callbacks=[tensorboard_callback]
  )
  return model

def predict(model_trained, X_test, y_test, scoring=accuracy_score):
  
  y_test_norm = np.argmax(y_test, axis=1)
  y_pred_prob = model_trained.predict(X_test)
  y_pred = np.argmax(y_pred_prob, axis=1)

  return scoring(y_test_norm, y_pred)

IndentationError: ignored

In [0]:
def get_cnn_v5(input_shape, num_classes):
  return Sequential([
  Conv2D(filters=32, kernel_size=(3,3), activation='relu', input_shape=input_shape),
  Conv2D(filters=32, kernel_size=(3,3), activation='relu', input_shape=input_shape),
  Conv2D(filters=32, kernel_size=(3,3), activation='relu', padding='same'),
  MaxPool2D(),
  Dropout(0.3),
  
  Conv2D(filters=64, kernel_size=(3,3), activation='relu', padding='same'),
  Conv2D(filters=64, kernel_size=(3,3), activation='relu'),
  MaxPool2D(),
  Dropout(0.3),

  Conv2D(filters=64, kernel_size=(3,3), activation='relu', padding='same'),
  Conv2D(filters=64, kernel_size=(3,3), activation='relu'),
  MaxPool2D(),
  Dropout(0.3),

  Flatten(),

  Dense(1024, activation='relu'),
  Dropout(0.3),

  Dense(1024, activation='relu'),
  Dropout(0.3),

  Dense(num_classes, activation='softmax')
  ])

In [14]:
model = get_cnn_v5(input_shape, num_classes)
model_trained = train_model(model, X_train, y_train)

predict(model_trained, X_test, y_test)

Epoch 1/5
272/272 [==============================] - 6s 21ms/step - loss: 2.6348 - accuracy: 0.2729 - val_loss: 1.2389 - val_accuracy: 0.6328
Epoch 2/5
272/272 [==============================] - 5s 19ms/step - loss: 0.9503 - accuracy: 0.7011 - val_loss: 0.3896 - val_accuracy: 0.8721
Epoch 3/5
272/272 [==============================] - 5s 19ms/step - loss: 0.4680 - accuracy: 0.8531 - val_loss: 0.1557 - val_accuracy: 0.9513
Epoch 4/5
272/272 [==============================] - 5s 20ms/step - loss: 0.3160 - accuracy: 0.9032 - val_loss: 0.0936 - val_accuracy: 0.9725
Epoch 5/5
272/272 [==============================] - 5s 19ms/step - loss: 0.2264 - accuracy: 0.9316 - val_loss: 0.0557 - val_accuracy: 0.9846


0.9582766439909297

In [20]:
model_trained.evaluate(X_test, y_test)

138/138 [==============================] - 1s 4ms/step - loss: 0.1848 - accuracy: 0.9583


[0.1847628504037857, 0.958276629447937]

In [0]:
def get_model(params):
  return Sequential([
  Conv2D(filters=32, kernel_size=(3,3), activation='relu', input_shape=input_shape),
  Conv2D(filters=32, kernel_size=(3,3), activation='relu', padding='same'),
  MaxPool2D(),
  Dropout(params['dropout_cnn_block_one']),
  
  Conv2D(filters=64, kernel_size=(3,3), activation='relu', padding='same'),
  Conv2D(filters=64, kernel_size=(3,3), activation='relu'),
  MaxPool2D(),
  Dropout(params['dropout_cnn_block_two']),

  Conv2D(filters=128, kernel_size=(3,3), activation='relu', padding='same'),
  Conv2D(filters=128, kernel_size=(3,3), activation='relu'),
  MaxPool2D(),
  Dropout(params['dropout_cnn_block_three']),

  Flatten(),

  Dense(1024, activation='relu'),
  Dropout(params['dropout_dense_block_one']),

  Dense(1024, activation='relu'),
  Dropout(params['dropout_dense_block_two']),

  Dense(num_classes, activation='softmax')
  ])

In [0]:
def func_obj(params):
  model = get_model(params)
  model.compile(loss= 'categorical_crossentropy', optimizer='Adam', metrics=[ 'accuracy' ])
  
  model.fit(
      X_train, 
      y_train,
      batch_size=int(params.get('batch size', 128)),
      epochs=5,
      verbose=0
  )

  score = model.evaluate(X_test, y_test, verbose=0)
  accuracy = score[1]
  print(params, 'accuracy = {}'.format(accuracy))

  return {'loss': -accuracy, 'status': STATUS_OK, 'model': model} 
  #the function wants to minimize. accuracy attempts to be "1". to combine it, the accuracy must be presented as going to "-1".

In [23]:
space = {
    'batch size' : hp.quniform('batch size', 100, 200, 10),
    'dropout_cnn_block_one': hp.uniform('dropout_cnn_block_one', 0.3, 0.5),
    'dropout_cnn_block_two': hp.uniform('dropout_cnn_block_two', 0.3, 0.5),
    'dropout_cnn_block_three': hp.uniform('dropout_cnn_block_three', 0.3, 0.5),
    'dropout_dense_block_one': hp.uniform('dropout_dense_block_one', 0.3, 0.7),
    'dropout_dense_block_two': hp.uniform('dropout_dense_block_two', 0.3, 0.7)
}

best = fmin(
    func_obj,
    space,
    tpe.suggest,
    30,
    Trials()
)

{'batch size': 110.0, 'dropout_cnn_block_one': 0.33484307421691667, 'dropout_cnn_block_three': 0.4391162947721748, 'dropout_cnn_block_two': 0.48338501879906703, 'dropout_dense_block_one': 0.5842250020436467, 'dropout_dense_block_two': 0.6550915490629139}
accuracy = 0.8623582720756531
{'batch size': 100.0, 'dropout_cnn_block_one': 0.412895091768431, 'dropout_cnn_block_three': 0.45594274751809305, 'dropout_cnn_block_two': 0.4880232264378016, 'dropout_dense_block_one': 0.5468576595938822, 'dropout_dense_block_two': 0.47152326804912686}
accuracy = 0.9034013748168945
{'batch size': 160.0, 'dropout_cnn_block_one': 0.3437162454081508, 'dropout_cnn_block_three': 0.40107824798503033, 'dropout_cnn_block_two': 0.35022370085068477, 'dropout_dense_block_one': 0.6063206176414275, 'dropout_dense_block_two': 0.5274056610631496}
accuracy = 0.9344671368598938
{'batch size': 180.0, 'dropout_cnn_block_one': 0.4584045610217915, 'dropout_cnn_block_three': 0.4848629133588705, 'dropout_cnn_block_two': 0.41274